## TP2 M1 Images : Clonage par édition de Poisson

**ENS Paris-Saclay**

**Département de Mathématiques**



---


**NOMS et Prénoms:**      Corentin Cornou, Marianne Déglise

---


## Consignes pour le rapport de TP

Ce TP comporte diverses questions. Les réponses, les résultats des expériences
(images, figures), ainsi que le code devront être contenus dans ce
notebook.

Il est possible de travailler en groupes de deux élèves (pas plus) ou individuellement.

Vous êtes également invité à rajouter toutes remarques ou commentaires sur les
résultats de votre code là où vous le jugerez nécessaire. Vous pouvez créer
une nouvelle cellule en cliquant sur l'icone **+ Code**. **Pensez à sauvegarder
régulièrement votre travail** (Fichier -> Enregistrer ou CTRL+S).

Le TP doit être soumis sur eCampus dans un délai de **deux semaines au plus tard**. \
 <font size="4"> <b>Une pénalité de <font color=red>deux points par
 jour</font> de retard sera appliquée </b>à partir de ce moment-là</font>. \
Le fichier joint doit être un notebook exécutable au format *ipynb* (dans
*Google Colab*, Fichiers -> Télécharger le fichier ipynb), **intitulé
nom_prenom_TP2.ipynb. Veuillez vous en assurer.**

Avant d'envoyer votre TP, nous vous recommandons de cliquer sur
Exécution -> Tout exécuter.  Vérifier ensuite que l'ensemble du notebook ne
comporte pas d'erreurs.

<font color=red><b>Important</b></font> : si le travail est effectué en groupe, **indiquer les noms des deux membres du groupe**. En plus, il ne faut soumettre qu'un seul foi le notebook dans eCampus pour le group, <b>jamais individuellement deux fois le même</b>.

# Mise en route

Ce TP porte sur l’équation de Poisson et ses applications au traitement d’image. Dans ce TP, nous étudirons l'article [Poisson Image Editing](http://www.ipol.im/pub/art/2016/163/).

## Modules Python et fichiers nécessaires

On commence par charger les modules python nécessaires pour l'exécution du TP.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
import skimage.io
import scipy.sparse as sp

plt.show()
plt.ion()

%matplotlib inline

In [ ]:
OperatorMatrix = np.ndarray[np.ndarray[float]]
Image = np.ndarray[np.ndarray[float]]
VectorField = np.ndarray[np.ndarray[float]]

In [ ]:
def plot_img(x:Image)-> None:
    """
    Affiche une image passée en entrée.
    """
    assert np.all(x >= 0) and np.all(x <= 1), "Les valeurs de l'image doivent être entre 0 et 1!"

    plt.imshow(x, vmin=0, vmax=1, interpolation="nearest", cmap='gray')
    plt.axis('off')
    plt.show()

# Introduction

Dans ce TP, on étudie le cas "*seamless cloning*" de l'article [Poisson Image Editing](http://www.ipol.im/pub/art/2016/163/). L'idée est de copier une partie d'une image source `source` dans une image cible `dest`.
Le script suivant charge l'image cible `dest`, l'image source `source` et le masque `mask`. `dest` est l'image originale sur laquelle on va copier-coller une partie de l'image `source`. Le masque `mask` est une image binaire (les intensités sont 0 ou 1), ne valant 1 qu'à l'endroit où on veut faire le copié collé.

🔴 Visualiser les images `source`, `dest` et le masque `mask` à l'aide la fonction `plot_img`.

In [ ]:
import urllib, os

def load_img(path  :str)-> Image:
    if path.startswith("http://"):
        name = path.split('/')[-1]

        if not os.path.exists(name):
            urllib.request.urlretrieve(path, name)

        return plt.imread(name)

    return plt.imread(path)


source = load_img("http://gabarro.org/img/poisson_source_gray.png")
dest = load_img("http://gabarro.org/img/poisson_dest_gray.png")
mask = load_img("http://gabarro.org/img/poisson_trimap.png")

<br> <div> <center> <img src="https://gfacciol.github.io/afh/TP_M1_Images/TP2/copie-colle.jpg" width="500"/> </center> </div> <br>

🔴 **Question 1 :** Utiliser la fonction `naive_copypaste` définie ci-dessous pour effectuer un simple copié-collé de l'image source vers l'image cible, correspondant à la zone contenue dans le masque, comme illustré à la figure ci-dessus. Commenter le résultat. Est-il conforme à vos attentes ? Vous convient-il ?

In [ ]:
def naive_copypaste(source : Image, dest :Image, mask : "np.ndarray[np.ndarray[bool]]") -> Image:
    """
    Effectue un copier-coller naïf en copiant la source dans la destination selon le masque.
    Args:
    - source (np.array): Image source
    - dest (np.array): Image destination
    - mask (np.array): Tableau contenant des booléens contenant le masque de copier-coller
    """
    return mask * source + (1. - mask) * dest

copie_naive = naive_copypaste(source, dest, np.round(mask))
plot_img(copie_naive)

🔥 **Réponse :** Le résultat n'est pas encore très satisfaisant car on voit la démarcation de la zone copiée. On veut applatir le fond pour avoir une transition invisible.

*🔴* **Question 2 :** Pour quel type d'images obtiendrait-t-on de très bon résultats avec un simple copié-collé ? Inversement, quel cas vous semble les plus défavorables ?

🔥 **Réponse :** Une image avec un fond parfaitement uniforme, ou un png archeraient très bien. En revanche une photo avec des variations de couleurs fréquentes (un fond non uniforme, comme un coucher de soleil par exemple) archera beaucoup moins.

# Édition de Poisson

L'**édition de Poisson** consiste à faire du traitement d'image, non pas à partir des valeurs d'intensités des pixels, mais plutôt avec la valeurs de gradients d'intensité.
Plusieurs manipulations peuvent être effectuées sur les gradients. Par exemple, on peut remplacer certains morceaux de gradients d'une image A par ceux d'une image B. On peut aussi annuler tout ou partie des gradients, *etc*. Après modification des gradients, on peut récupérer une image en résolvant l'équation de Poisson.

On peut distinguer deux façons de résoudre (ou de formuler) l'équation de Poisson :
- résolution *globale*
- résolution *locale*

On résout l'équation de Poisson *globale* lorsque l'on a édité (modifié) le domaine de l'image dans son intégralité. Comme vous l'avez vu en cours, une façon très simple et pourtant très efficace consiste à utiliser la TFD (dans le domaine des fréquences).
A l'inverse, lorsqu'on a modifié une partie de l'image seulement, on résout l'équation de Poisson *locale*, en écrivant explicitement le système linéaire (comme pour la méthode des différences finies vue dans le cours d'EDP en L3).

Dans ce TP, nous allons étudier ces deux cas de figure, dans le cas du "*seamless cloning*" afin d'améliorer les résultats que vous avez obtenus dans l'introduction.

## Résolution *globale*

Comme annoncé précédement, ce premier cas de figure se résout très simplement lorsqu'on traduit le problème dans le domaine fréquentiel ("en *Fourier*").

La méthode a été proposée et vue en cours. L'objectif du préambule suivant est de la résumer en quelques phrases. Répondez à ce préambule pour vous remémorer le cours.

🔴 **Question 3** : Cette astuce nécessite que le domaine d'édition soit un rectangle. Pourquoi ? Est-ce un inconvéniant ? (répondre de manière très succincte)

🔥 **Réponse :** Les bords de l'image doivent correspondre aux coordonnées 2D. Ce n'est pas un problème, on peut augmenter le domaine pour le rendre rectangulaire.

L'idée est de résoudre l'équation de Poisson en passant dans le domaine fréquentiel, discret (avec la librairie `np.fft`). L'image est alors vue comme un polynôme trigonométrique à deux variables
$$ u(x,y)=\sum_{m=-N/2}^{N/2}\sum_{n=-N/2}^{N/2}u_{m,n} ~ e^{2i\pi(mx+ny)} $$

🔴 **Question 4** Justifier l'existence du Laplacien $\Delta u$ et écrire son expression en $(x,y)$.

🔥 **Réponse :** La fonction est $C^{\infty}$ donc le laplacien existe. On a : $\Delta u = \sum_{m=-N/2}^{N/2}\sum_{n=-N/2}^{N/2} -((2 \pi m)^2 + (2 \pi n)^2) ~ u_{m,n} ~ e^{2i\pi(mx+ny)} $

🔴 **Question 5** À partir de l'expression de $u(x, y)$ et $\Delta u(x, y)$, déduire comment on peut résoudre l'équation de Poisson globale.

⚠️ Vous aurez soin d'expliquer le cas $(m, n) = (0, 0)$ !

🔥 **Réponse :** On peut résoudre l'équation de Poisson par une identification des coefficients de Fourier : Si le champ de vecteurs est $\mathbf{V} = (V^x, V^y)$ (en fourier), on a pour tout couple $(m,n)$: $$((2 \pi m)^2 + (2 \pi n)^2) ~ \widehat{u_{m,n}} = 2i \pi m \widehat{V^x_{m,n}} + 2i \pi n \widehat{V^y_{m,n}}$$
Donc en particulier pour tous $(m,n)\neq (0,0)$, 
$$\boxed{\widehat{u_{m,n}} = \frac{2i \pi m \widehat{V^x_{m,n}} + 2i \pi n \widehat{V^y_{m,n}}}{((2 \pi m)^2 + (2 \pi n)^2)}}$$
On a un degré de liberté supplémentaire car le problème de minimisation est invariant lorsqu'on rajoute une constante. On peut donc choisir $\widehat{u_{0,0}}$ pour obtenir la moyenne de l'image que l'on souhaite avoir.

Vous allez maintenant implémenter la résolution globale de l'équation de Poisson. Vous n'utiliserez ici que des techniques d'analyse de Fourier.

🔴 **Question 6 :** En reprenant l'expression de $u(x, y)$, écrire une fonction `gradient` qui calcule $$ \left( \frac{\partial u}{\partial x}, \frac{\partial u}{\partial y} \right)$$
à l'aide de la Transformée de Fourier.

In [ ]:
def gradient(u : Image)-> VectorField:
    """
    Calcule le gradient de l'image `u` à l'aide de la Transformée de Fourier.
    Args:
    - u (np.array): Un tableau numpy contenant l'image.
    Returns:
    - grad_x (np.array): Un tableau numpy de la même dimension que `u` qui contient le gradient selon x.
    - grad_y (np.array): Un tableau numpy de la même dimension que `u` qui contient le gradient selon y.
    """

    tf = np.fft.fft2(u)

    freq_x = np.fft.fftfreq(tf.shape[0])
    freq_y = np.fft.fftfreq(tf.shape[1])

    freq_x, freq_y = np.meshgrid(freq_x, freq_y)

    # VOTRE CODE ICI
    grad_x = np.fft.ifft2(2 * 1j * np.pi * freq_x.T * tf)
    grad_y = np.fft.ifft2(2 * 1j * np.pi * freq_y.T * tf)

    return np.real(grad_x), np.real(grad_y)


*🔴* **Question 7 :** Calculer le champ de gradient $ \left( \frac{\partial w}{\partial x}, \frac{\partial w}{\partial y} \right)$ de $w$ tel que le gradient de $w$ vaut celui de l'image `dest` là où `mask` vaut `0` et celui de l'image `source` là où `mask` vaut `1`. On s'aidera de la fonction `naive_copypaste`.

In [ ]:
gradx_source, grady_source = gradient(source)
gradx_dest, grady_dest = gradient(dest)
gradx_w = naive_copypaste(gradx_source, gradx_dest, np.round(mask))
grady_w = naive_copypaste(grady_source, grady_dest, np.round(mask))
grad_w = (gradx_w, grady_w)
print(np.max(gradx_dest - np.gradient(dest, axis = 0)))

🔴 **Question 8 :** Le champ de gradient ainsi créé contient les gradients de l'image cible ou source selon l'endroit où l'on veut faire le clonage. Quel est le rapport entre le clonage et la résolution d'une équation de Poisson ? En d'autres termes, pourquoi résoudre une équation de Poisson permet-il de résoudre le problème du clonage ?

🔥 **Réponse :** Dans cette méthode on copie les gradients plutôt que les pixels eux-même afin de copier les formes, mais d'avoir une cohérence sur les couleurs grâce à l'équation de poisson et aux conditions au bord.

*🔴* **Question 9 :** Calculer le Laplacien de l'image clonée à partir du champ de gradient précédemment calculé. Vous pouvez vous aider de la fonction `gradient` définie précédemment.

In [ ]:
def laplacien(gradx,grady) : return gradient(gradx)[0] + gradient(grady)[1]
lap_w = laplacien(gradx_w, grady_w)

*🔴* **Question 10 :** Écrire une fonction qui inverse le Laplacien d'une image donnée. On souhaite avoir une image clonée de moyenne 140.

In [ ]:
def inverse_laplacien(lap :VectorField, mean :float=140/255) -> Image:
    """
    Calcule l'inverse du Laplacian donné.
    Args:
    - lap (np.array): Tableau numpy à deux dimensions qui contient le Laplacien.
    - mean (float): La moyenne désirée pour l'image de sortie.
    """
    H,W = lap.shape
    tf_lap = np.fft.fft2(lap)
    freq_x = np.fft.fftfreq(H)
    freq_y = np.fft.fftfreq(W)

    freq_x, freq_y = np.meshgrid(freq_x, freq_y)

    # Inversion du Laplacien
    denom = -(2 * np.pi)**2 * (freq_x * freq_x + freq_y * freq_y)
    denom[0, 0] = 42

    itf_lap = tf_lap / denom.T

    # Corriger la moyenne
    itf_lap[0,0] = mean * H * W

    # Retour dans le domaine original
    output = np.fft.ifft2(itf_lap).real

    assert abs(output.mean() - mean) < 1e-4, "La moyenne n'a pas été bien modifiée !"

    return np.clip(output,0,1) # compense les erreurs de calul

*🔴* **Question 11 :** Appliquer la fonction `inverse_laplacien` sur le Laplacien trouvé à la question 4. Visualiser l'image obtenue et commenter le résultat. En particulier, comparer avec le résultat obtenu par un simple copié-collé.

In [ ]:
copie = inverse_laplacien(lap_w)

plot_img(copie)
plot_img(copie_naive)

🔥 **Réponse :** Le résultat obtenu par `inverse_laplacien`est bien meilleur que celui obtenu par un simple copier/coller. On ne discerne en effet plus les bords de la partie qui a été dupliquée mais l'aigle ainsi que le reste de l'image destination sont parfaitement conservés par l'algorithme.

## Résolution *locale*

L'équation de Poisson $\Delta u = f$ peut être résolue avec des techniques de différences finies. On peut discrétiser l'espace et utiliser la discrétisation du Laplacien à cinq points introduite en cours.

Lorsqu'on y rajoute des conditions de bord, on obtient un système linéaire. Le système linéaire est très simple à résoudre explicitement, ou numériquement en utilisant n'importe quel *solver* standard (en *Python*, on pourra utiliser par exemple `scipy.sparse.linalg.spsolve`). La principale difficulté est d'écrire explicitement la matrice du système à résoudre (il faut faire très attention à ne pas introduire une erreur d'indice par exemple). La seconde difficulté est qu'il est très coûteux de résoudre ce système lorsque la région à cloner est grande.

L'objectif des questions suivantes est d'écrire correctement le système linéaire et de le résoudre par un *solver*. Les différentes questions vous guident progressivement.

🔴 **Question 12 :** Les *solvers* résolvent des problèmes du type $Ax=b$, où l'inconnue $x \in \mathbb{R}^n$, $A$ est la matrice du système, $A \in \mathcal{M}_{n}(\mathbb{R})$ et $b \in \mathbb{R}^n$ est le second membre.

La première chose pour simplifier est de travailler en 1D, c'est-à-dire que l'on va transformer les images 2D en vecteur 1D. Vectoriser les images `source`, `dest` et le masque `mask` précédents (vous pouvez utiliser la méthode `.flatten()` de numpy. Attention, vous aurez soin au préalable de conserver en mémoire la taille initiale des images 2D, afin de pouvoir retransformer les vecteurs en images (avec la méthode `.reshape()` de numpy).

In [ ]:
# Conserver la taille des images
H, W = source.shape

# Vectoriser les images
source = source.flatten()
dest = dest.flatten()
mask = mask.flatten()

La deuxième simplification du problème consiste à discrétiser les opérateurs différentiels comme le gradient, la divergence  et le Laplacien. Pour cela, on va considérer ces opérations linéaires sous leur forme matricielle. On vous donne le code de la fonction `gradient_discret` qui prend en argument les dimensions `H` et `W` et qui renvoie la matrice 2D de différence finie qui correspond à la discrétisation du gradient.

In [ ]:
def discrete_gradient(H :int, W: int) -> OperatorMatrix:
    """
    Calcule la matrice associée au calcul du gradient.
    Args:
    - H (int): La hauteur de l'image sur laquelle on applique le gradient.
    - W (int): La largeur de l'image sur laquelle on applique le gradient.
    Returns:
    - La concaténation selon l'axe des lignes de deux matrices:
    -- Un de taille ((H - 1) * W, H * W) correspondant au gradient selon x
    -- Un de taille (H * (W - 1), H * W) correspondant au gradient selon y
    """

    x = sp.eye(W - 1, W, 1) - sp.eye(W - 1, W)
    y = sp.eye(H - 1, H, 1) - sp.eye(H - 1, H)

    # Calcule le gradient selon x, de taille ((H - 1) * W, H * W)
    p = sp.kron(sp.eye(H), x)

    # Calcule le gradient selon y, de taille (H * (W - 1), H * W)
    q = sp.kron(y, sp.eye(W))

    return sp.vstack([p, q])

*🔴* **Question 13 :** Notez que les matrices de discrétisation des autres opérateurs différentiels évoqués plus haut s'obtiennent à partir de celle du gradient que vous venez d'implémenter. En effet, pour rappel, la divergence est l'opposée de l'adjoint du gradient, et le Laplacien est la divergence du gradient. Écrire une fonction `divergence_discret` et une fonction `laplacien_discret` qui calcule respectivement les matrices de discrétisation de la divergence et du Laplacien. Ces fonctions prendront en arguments les dimensions `H` et `W` et seront basées sur la fonction `gradient_discret` précédente.

In [ ]:
def divergence_discret(H : int, W: int)->OperatorMatrix:
    """
    Calcule la matrice associée au calcul de la divergence.
    Args:
    - H (int): La hauteur de l'image sur laquelle on applique la divergence.
    - W (int): La largeur de l'image sur laquelle on applique la divergence.
    """
    return -discrete_gradient(H,W).H

def laplacien_discret(H : int, W :int)-> OperatorMatrix:
    """
    Calcule la matrice associée au calcul du Laplacien.
    Args:
    - H (int): La hauteur de l'image sur laquelle on applique le Laplacien.
    - W (int): La largeur de l'image sur laquelle on applique le Laplacien.
    """
    grad = discrete_gradient(H,W)
    return - grad.H @ grad #ne pas avoir à calucler deux fois la fonction le gradient permet de gagner du temps  

*🔴* **Question 14 :** Soit $v$ l'image à copier-coller sur l'image cible $g$ et $\Omega$ le domaine à cloner (de l'image source vers l'image cible). Le problème que l'on cherche à résoudre s'écrit dans le domaine continu (c'est l'équation (5.1) du poly):

\begin{cases}
\Delta u = \Delta v & \mathrm{in}\ \Omega \\
      u=g & \mathrm{in}\ \Omega^c
\end{cases}

Écrire en $\LaTeX$ l'équation ci-dessus dans le domaine discret. Pour modéliser $\Omega$ ou $\Omega^c$, vous utiliserez la matrice $M$ qui est la matrice diagonale du masque vectorisé et la matrice $L$ du Laplacien discret.

🔥 **Réponse :** *Écrivez votre réponse ici :*
\begin{cases}
  MLu = MLv \\
  (I-M)u = (I-M)g
\end{cases}

🔴 **Question 15 :** Maintenant que vous l'avez écrit en langage mathématique, l'objectif de cette question est d'écrire exactement la même chose en utilisant les matrices et tableaux précédents `Lw`, `v`, *etc.* Écrire la matrice $A$ et le second membre $b$ du système. Vous aurez besoin d'une matrice de masque, diagonale et de la matrice identité.

**Astuces :** La fonction `diags` de `scipy.sparse` prend en argument un vecteur `x` et renvoie la matrice diagonale dont les coefficients $x_i$ sur la diagonale. La fonction `eye(n,n)` de `scipy.sparse` renvoie la matrice $I_n$.

**Attention :** N'oubliez pas que les images de taille $(H, W)$ ont été vectorisées ! Ce qui signifie que le système est de taille $(H \times W)^2$

In [ ]:
L = laplacien_discret(H, W)[:H*W, :H*W]     # Laplacien discret
M = sp.diags(mask.flatten())    # Opérateur associé au masque
I = sp.eye(H * W, H * W)        # Opérateur associé à l'identité
M  = np.round(M)

A = (I - M) @ I - M @ L
b = (I - M) @ dest- M @ L @ source


🔴 **Question 16 :** Résoudre le système $Ax = b$.

**Astuce :** La fonction `spsolve` de la librairie `scipy.sparse.linalg` appliquée à un array numpy $A$ de taille $n \times n$ et à un array numpy $b$ de taille $n$ renvoie un array numpy $x$ solution de $Ax = b$.

In [ ]:
from scipy.sparse.linalg import spsolve
solution = spsolve(A,b) 


🔴 **Question 17 :** Vous avez presque fini ! Il ne reste plus qu'à remettre en forme le vecteur $x$ de taille $(H \times W,)$ en un tableau 2D de taille $(H, W)$ en utilisant la méthode `.reshape()`. Former l'image clonée et la visualiser. Commenter le résultat obtenu.

In [ ]:
final_image = np.clip(solution.reshape(H,W),0,1)

plot_img(np.clip(final_image,0,1)) #compense les erreurs de calculs
plot_img(copie)
...

🔥 **Commentaire :** Le résultat ainsi obtenu est tout à fait satisfaisant, on n'aperçoit pas les bordures du masque et le rapace ainsi que le fond sont conservés. De plus, il semblerait que cette version conserve un contraste légèrement meilleur que celle appliquée en partie 1.

## Bonus

In [ ]:
#On commence par restructuer les images "applaties" en partie 2
source = source.reshape(H,W)
dest = dest.reshape(H,W)
mask = mask.reshape(H,W)

🔴 **Question B1 :** Dans ce TP, vous avez résolu le problème de trouver $u$ vérifiant :
\begin{cases}
\Delta u = f & \mathrm{in}\ \Omega \\
      u=g & \mathrm{in}\ \Omega^c,
\end{cases}
où $f$ est le gradient cible. Ceci correspond à remplacer le gradient de l'image source brusquement par le gradient de l'image cible, et est appelé "remplacement" dans l'article [Poisson Image Editing](http://www.ipol.im/pub/art/2016/163/). Néanmoins, dans cet article, dans la section 4, les auteurs considèrent plusieurs options pour le gradient cible :
-  une **moyenne** des gradients des images source et cible
-  le **maximum** (pixel à pixel) des gradients des images source et cible

Reprenez le code écrit dans la partie "Résolution *globale*". Modifiez-le pour implémenter ces deux options (vous n'avez qu'à faire un copié-collé des cellules des la partie "Résolution *globale* et modifier une ou deux lignes). Tester ces deux options. Qu'en pensez-vous ?  Si vous êtes curieux et avez du temps, vous pouvez aussi tester la somme des gradients des images source et cible.

In [ ]:
###    Copier-coller le contenu des cellules de la partie Résolution globale ici.   ###

###    Modifier les lignes qui définissent le gradient cible pour implémenter la moyenne  ###
Wx = (gradx_source + gradx_dest) / 2
Wy = (grady_source + grady_dest) / 2

gradx_W = naive_copypaste(Wx, gradx_dest, mask)
grady_W = naive_copypaste(Wy, grady_dest, mask)

lap_W = laplacien(gradx_W, grady_W)

moyenne = inverse_laplacien(lap_W)

plot_img(moyenne)

In [ ]:
###    Copier-coller le contenu des cellules de la partie Résolution globale ici.     ###

###    Modifier les lignes qui définissent le gradient cible pour implémenter le max  ###
Wx = gradx_source * np.abs(gradx_source > gradx_dest) + gradx_dest * (np.abs(gradx_source) <= gradx_dest)
Wy = grady_source * np.abs(grady_source > grady_dest) + grady_dest * np.abs(grady_source <= grady_dest)

gradx_W = naive_copypaste(Wx, gradx_dest, mask)
grady_W = naive_copypaste(Wy, grady_dest, mask)

lap_W = laplacien(gradx_W, grady_W)

maximum = inverse_laplacien(lap_W)

plot_img(maximum)

In [ ]:
Wx = (gradx_source + gradx_dest)
Wy = (grady_source + grady_dest)

gradx_W = naive_copypaste(Wx, gradx_dest, mask)
grady_W = naive_copypaste(Wy, grady_dest, mask)

lap_W = laplacien(gradx_W, grady_W)

somme = inverse_laplacien(lap_W)

plot_img(np.clip(somme,0,1))

In [ ]:
plot_img(copie)
plot_img(moyenne)
plot_img(maximum)
plot_img(somme)
plot_img(final_image)

🔥 **Commentaire :** Les deux premières méthodes proposées sont peu satisafaisantes (en tous cas sur l'exemple). En effet, la moyenne pâlit considérable l'oiseau qui apparait binen moins vivement que dans la méthode de base. Quant à la méthode du max elle apparaître d'importantes aberrrations en plus de rendre l'oiseau moins net.
La somme enfin présente un intérêt car elle présente une version qui semble légèrement plus contrastée que la version initale (peut-être même légèrement plus que la version locale).

🔴 **Question B2 :** Vous disposez d'une image avec des régions très sombre que vous souhiatez éclaircir. Comment pouvez-vous utiliser l'édition de Poisson pour y parvenir ?

**Indice :** Lire la partie 4 de l'article [Poisson Image Editing](http://www.ipol.im/pub/art/2016/163/), paragraphe "Contrast enhancement". Essayez de comprendre la partie "Contrast enhancement" et résumez là en quelques phrases.

🔥 **Réponse :** On utilise ici une seule image. On définit la zone à remplacer $\Omega$ comme étant l'ensemble des points de l'image dont la luminosité est inférieure à un certain seuil fixé. On définit alors le champ gradient servant à la résolution comme $\mathbf{v} = \alpha \nabla I$ sur la zone $\Omega$ , et $\nabla I$ en dehors, où $I$ est l'image originale et $\alpha$ est un coefficient constant décidant à quel point on éclaircit.

🔴 **Question B3 :** Si vous avez bien compris la partie "Contrast Enhancement", vous voyez qu'elle est très simple à implémenter. Comme à la question Bonus-1, il suffit de reprendre le code de la partie Résolution globale par exemple et de modifier très simplement une ou deux ligne. Vous pouvez essayer cette méthode si vous le souhaitez. L'image blablabla.png contient une zone sombre située dans la partie [x:x+??, y:y+??].

In [ ]:
def simplest_color_balance(img : Image, s1 : float, s2 :float) -> Image:
    sorted_values = [x for ligne in img for x in ligne]
    sorted_values.sort()

    N = len(sorted_values)
    i1, i2 = round(N * s1), round(N * (1 - s2) - 1)
    vmin = sorted_values[i1]
    vmax = sorted_values[i2]

    saturated_img = np.copy(img)
    n,m = saturated_img.shape
    saturated_img = np.vectorize(lambda x : min(vmax,max(vmin,x)))(saturated_img)

    if vmin < vmax :
      return (saturated_img - vmin) / (vmax - vmin)
    else : 
      return saturated_img - vmin

def colorize(grey_scale_function : "callable[[Image],...]") -> "callable[[Image], ...]":
    """
    Transforme une fonction prenant en entrée une image en échelle de gris, en une autre prenant en entrée une image en couleur et appliquant la fonction
    en échelle de gris sur chaque composante de l'image RBG
    """
    def fun(img : Image) :
        split_RGB = (img[:,:,i] for i in range(3))
        processed_split_RGB = tuple(map(grey_scale_function, split_RGB))
        return np.dstack(processed_split_RGB)
    return fun

simplest_color_balance_RGB = colorize(lambda x : simplest_color_balance(x,.01,.01))

In [ ]:
#u = load_img("https://gfacciol.github.io/afh/TP_M1_Images/TP2/dark_cat.png")
u = load_img("dark_cat.png")

mask = np.zeros(u.shape)
mask[:,:] = 1
I = (u[:,:,0] + u[:,:,1] + u[:,:,2])/3
alpha = 5 # facteur multiplicatif pour l'éclaircissement.
gradx_I, grady_I = gradient(I)
IWx = gradx_I * alpha
IWy = grady_I * alpha

lap_I = laplacien(IWx, IWy)

eclair = inverse_laplacien(lap_I)
v= u.copy()
for i in range(u.shape[0]):
    for j in range(u.shape[1]):
        v[i,j] *= eclair[i,j]

plot_img(np.clip(u,0,1))
plot_img(np.clip(v,0,1))


plot_img(simplest_color_balance_RGB(u))
plot_img(np.clip(simplest_color_balance_RGB(v),0,1))

🔴 **Question B4 :** Comment feriez-vous pour appliquer l'édition de Poisson pour une image en couleur ?

🔥 **Commentaire :** On applique Poisson sur l'image grisée, puis on rétablit les couleurs (comme ci-dessus par exemple).